<a href="https://colab.research.google.com/github/ftvalentini/misc-notebooks/blob/master/borges_hipalages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buscando hipálages en los cuentos de Borges con NLP
#### Autores: Julián Martínez Correa y Francisco Valentini

En Twitter alguién sugirió recopilar todas las hipálages de la literatura de Borges y [@manuelaristaran sugirió usar alguna técnica de Natural Language Processing](https://twitter.com/manuelaristaran/status/1249775645929934851) para esta tarea.  
Una hipálage es una combinación de palabras en la que se le atribuye a un sustantivo una acción (verbo) o una cualidad (adjetivo) que son propios de otro. Por ejemplo, la expresión "noche unámine" en el Las Ruinas Circulares.  
Siguiendo la sugerencia, decidimos abordar de forma exploratoria esta tarea con los siguientes pasos:  
- Restringimos el análisis a pares de palabras sustantivo-adjetivo y a los cuentos de Ficciones **en inglés**
- Extraemos todos los pares sustantivo-adjetivo de los cuentos usando un [modelo de dependecy parsing pre-entrenado](https://stanfordnlp.github.io/stanza/depparse.html) que reconoce las dependencias gramáticales en cada oración
- Medimos las distancias coseno entre los [word embeddings GloVe](https://nlp.stanford.edu/projects/glove/) de cada par de palabras. Un word embedding es un vector de dimensión *n* que codifica el signifcado de una palabra en relación al resto de las palabras de un vocabulario.
- Rankeamos los pares según la distancia coseno y exploramos rankings alternativos en función de la inspección visual de los resultados. 
<!-- normaliza la distancia por la distancia promedio del sustantivo o adejtivo a los *n* adjetivos o sustantivos más cercanos. -->

La hipótesis es que hay una mayor probabilidad de encontrar hipálages en pares sustantivo-adjetivo con distancia alta entre sus embeddings que en pares con distancia baja. Esto se debe a que **en general** las palabras que conforman las hipálages tienen una similitud semántica baja o al menos no tienden a usarse en el mismo contexto. 

En este sentido, este análisis no nos permite estricamente encontrar hipálages (y aún menos evaluar la bondad de los resultados), sino más bien encontrar pares de sustantivo-adjetivos inusuales, llamativos o interesantes en relación al uso común de la lengua inglesa.

### Descargas y librerías

- Instalamos [Stanza](https://github.com/stanfordnlp/stanza/), la librería desarrollada por Stanford que vamos a usar para encontrar las dependencias gramaticales en los textos. La librería ofrece modelos pre-entrenados para múltiples idiomas para aplicar en el contexto de tareas de NLP (en nuestro caso, dependency parsing). 

In [0]:
!pip install stanza

- Descargamos `PyPDF2` para convertir *Ficciones* de formato PDF a un archivo de texto plano.  

In [0]:
!pip install PyPDF2

- Cargamos las librerías que vamos a necesitar para hacer el análisis.

In [0]:
import numpy as np
import pandas as pd
import os

import stanza

import nltk
from nltk.tokenize import sent_tokenize
import gensim.downloader as gensim_api

import PyPDF2 

- Descargamos los modelos de `stanza` para textos en inglés. Los modelos se guardan por defecto en `/root/stanza_resources`. Esta carga nos llevó poco tiempo porque usamos Google Colab,  pero en otro contexto puede llevar algunos minutos -- lo mismo vale para el resto de instalaciones o descargas.

In [0]:
### PUEDE TARDAR VARIOS MINUTOS ###
stanza.download('en')

- Cargamos los vectores [GloVe](https://nlp.stanford.edu/pubs/glove.pdf), word embeddings ajustados por Stanford. En particular, vamos a usar los vectores de mayor dimensión posible (300) y ajustados con la wikipedia en inglés.

In [0]:
def load_embeddings():
    """ Load GloVe Vectors
        Returns: array of size (vocab_size, embeddings_dim)
    """
    vectors = gensim_api.load("glove-wiki-gigaword-300")
    return vectors

In [0]:
### PUEDE TARDAR VARIOS MINUTOS ###
word_vectors = load_embeddings()

- Descargamos los modelos de tokenización `punkt` para separar textos en oraciones. Solo lo vamos a usar para visualizar el texto de forma más amigable.

In [0]:
## revisar si esto es necesario
nltk.download('punkt')

- Convertimos el pdf con los cuentos en un archivo de texto. Leemos este archivo y lo almacenamos en `texto` como una gran cadena de caracteres.

In [0]:
def pdf_to_txt(pdf_file, txt_file, pdf_first_page=10):
    """ Convert pdf_file to txt_file with strings
    """
    with open(pdf_file,'rb') as pdf, open(txt_file, 'w') as txt:
        read_pdf = PyPDF2.PdfFileReader(pdf_file)
        number_of_pages = read_pdf.getNumPages()
        for page_number in range(pdf_first_page-1, number_of_pages):  
            page = read_pdf.getPage(page_number)
            page_content = page.extractText()
            txt.write(page_content)

def read_text(file):
    """ Reads file as string
    file can be PDF or txt -- if it is PDF it first transforms to txt
    """
    file_root = os.path.splitext(file)[0]   
    file_ext = os.path.splitext(file)[1] 
    if file_ext == '.pdf':
        pdf_to_txt(file, file_root+'.txt')
    with open(file_root+'.txt', "r", encoding='utf-8') as f:
        texto = f.read() 
    return texto

In [101]:
%%time
input_file = 'borges_collected-fictions.pdf'
# input_file = 'the_library_of_babel.txt'
texto = read_text(input_file)

CPU times: user 6.99 s, sys: 17 ms, total: 7.01 s
Wall time: 7.01 s


### Limpieza del texto

Limpiamos el texto de la siguiente manera:  
- Reemplazamos los saltos de linea y cualquier otro carácter de tipo whitespace por espacios no repetidos porque no nos interesan los cambios de párrafo ni de página, sino solamente las oraciones  
- Reemplazamos el carácter Š por --

La documentación de `stanza` sugiere juntar varias oraciones en batches separando cada oración con dos saltos de linea `\n\n` para acelerar el procesamiento de muchos documentos. Sin embargo esto dificultaría rastrear al final del análisis las oraciones donde se encuentran las potenciales hipálages -- decidimos entonces conservar el texto como un solo string y que `stanza` se encargue de la tokenización.  

In [0]:
def clean_texto(texto):
    """Cleans raw text
       Returns: clean string
    """
    clean_text = " ".join(texto.split())
    clean_text = clean_text.replace("Š","—")
    return clean_text

texto = clean_texto(texto)

Veamos los primeros 500 caracteres del texto separados en oraciones para facilitar la visualización:

In [104]:
for s in sent_tokenize(texto[:500]):
    print(s)

A Universal History of Iniquity (1935) I inscribe this book to S.D.
— English, innumerable, and an Angel.
Also: I offer her that kernel of myself that I have saved, somehow— the central heart that deals not in words, traffics not with dreams, and is untouched by time, by joy, by adversities.
Preface to the First Edition The exercises in narrative prose that constitute this book were performed from 1933 to 1934.
They are derived, I think, from my re-readings of Stevenson and Chesterton, from the


### Dependencies parsing

Para realizar tareas de NLP con `stanza` es necesario construir un Pipeline con procesadores acordes al idioma de análisis.
El Pipeline se inicializa por defecto con un dependency parser (`depparse`) para determinar las dependencias sintácticas entre las palabras de una oración dada. Para poder correr este modelo es necesario procesar el texto de antemano con los siguientes procesadores:
- `tokenize`: separa el documento en oraciones
- `pos`: identifica el rol de cada palabra en la oración con un modelo de Part-of-Speech (POS) tagging pre-entrenado
- `lemma`: halla el lema correspondiente de cada palabra (la forma que representa las posibles formas flexionadas -- por ejemplo, "walk" es el lema de "walking")

Por defecto estos procesadores se ejecutan antes de `depparse`, el cual usa la información generada en los pasos precedentes para identificar las relaciones sintátictas.

In [91]:
%%time
nlp = stanza.Pipeline('en', verbose=False)
docs = nlp(texto)

CPU times: user 37min 24s, sys: 20.4 s, total: 37min 45s
Wall time: 37min 46s


Para identificar las relaciones sustantivo-adjetivo es necesario conservar las relaciones de tipo "amod" (*adjectival modifier*). Para cada relación identificada conservamos el índice de la oración para poder rastrearla más adelante.

In [0]:
def find_dependencies(doc):
    """Find amod dependencies in one parsed doc with one or more sentences 
    Returns: list of dependencies with (sentence_idx, head, dependent)
    """
    deps = list()
    for sent_idx, sent in enumerate(doc.sentences):
        id2word = {word.id: word.text for word in sent.words}
        deps += [(sent_idx, id2word[str(word.head)], word.text) \
                for word in sent.words if word.deprel=='amod']
    return deps
    
deps = find_dependencies(docs)

Veamos los tres primeros y los tres últimos pares hallados: 

In [106]:
print(deps[:3])
print(deps[-3:])

[(2, 'heart', 'central'), (4, 'prose', 'narrative'), (5, 'films', 'first')]
[(11110, 'layers', 'Everlasting'), (11115, 'aesthetics', 'literary'), (11115, 'others', 'many')]


## Word embeddings

Para cada par de palabras calculamos la distancia entre sus embeddings. Si no existe el word embedding para una palabra, no calculamos la distancia. Finalmente guardamos los resultados en un DataFrame. 

In [0]:
# palabras disponibles en GloVe
vocab = word_vectors.vocab.keys()

In [108]:
%%time
for i, dep in enumerate(deps):
    _words = dep[1:] 
    if set(_words).issubset(vocab):
        deps[i] = dep + (word_vectors.distance(*_words),)

CPU times: user 9min 53s, sys: 3.74 s, total: 9min 57s
Wall time: 9min 58s


In [133]:
deps_df = pd.DataFrame(deps, columns=['sent_idx', 'noun', 'adj', 'dist']) 
deps_df.sort_values(by=['dist'], inplace=True, ascending=False)
deps_df.dropna(inplace=True)
print(deps_df.shape)
print(deps_df.head(20))

      sent_idx          noun           adj      dist
4766      3853   schismatics           new  1.307657
9161      8804  trivialities           new  1.283182
891        651     anathemas         first  1.278724
5877      4975     spiderweb        better  1.277311
6931      5867      hecatomb        public  1.271595
6244      5276        karats          good  1.262450
4622      3721       paisano          same  1.247534
6223      5259  alexandrines          long  1.247089
3953      3152       impiety          near  1.246878
1728      1386       milonga          same  1.242564
7130      6084           men   punctilious  1.235335
9750      9680        series  misfortunate  1.232602
3815      2994          past    modifiable  1.231535
649        441        battle    indiscreet  1.231510
6350      5378        center     ineffable  1.231363
7135      6086     forewords          long  1.224867
2698      2058      hexagons        native  1.221834
1165       937    plastering        former  1.

In [0]:
print(deps_df.tail(20))

*DESCRIBIR RESULTADOS*

Conservamos aproximadamente tres cuartos de los pares de palabras con distancia más alta.

In [0]:
deps_df = deps_df.head(7500)

*describir: indicador normalizado por distancia a palabras mas cercanas para solucionar problemas del indicador anterior*
*creamos funcion para extraer distancia promedio a palabras mas cercanas* 
*explicar brevemente lo de upos*

In [0]:
def get_upos(word):
    """Get Universal POS of a word by itself (without sentence context) 
    """
    doc = nlp(word)
    upos = doc.sentences[0].words[0].upos
    return upos

def avg_distance_most_similar(word, tipo='ADJ', n=5, nsim=2000):
    """Return avg distance of a word to n most similar 'ADJ's or 'NOUN's
    tipo in ('ADJ','NOUN','ALL')
    """
    if word not in vocab:
        avg_distance = np.nan
        palabras = []
    else:
        most_similar = word_vectors.most_similar(word, topn=nsim)
        distancias = list()
        palabras = list()
        k = 0
        for i in most_similar:
            if tipo in ('ADJ','NOUN'):
                if get_upos(i[0]) == tipo:
                    distancias.append(1 - (i[1]+1) / 2)
                    palabras.append(i[0])
                    k += 1
            elif tipo == 'ALL':
                distancias.append(1 - (i[1]+1) / 2)
                palabras.append(i[0])
                k += 1
            if k == n:
                break
        avg_distance = sum(distancias) / len(distancias)
    return palabras, avg_distance

*explicar cada tipo de denominador*

In [0]:
%%time
# avg distance to adj/noun
deps_df[['noun_closest_adjs','noun_avg_dist_adjs']] = \
    deps_df.apply(lambda x: pd.Series(avg_distance_most_similar(x['noun'], tipo='ADJ', n=5)), axis=1)
deps_df[['adj_closest_nouns','adj_avg_dist_nouns']] = \
    deps_df.apply(lambda x: pd.Series(avg_distance_most_similar(x['adj'], tipo='NOUN', n=5)), axis=1)
# avg distance to any word
deps_df[['noun_closest_words','noun_avg_dist_words']] = \
    deps_df.apply(lambda x: pd.Series(avg_distance_most_similar(x['noun'], tipo='ALL', n=20)), axis=1)
deps_df[['adj_closest_words','adj_avg_dist_words']] = \
    deps_df.apply(lambda x: pd.Series(avg_distance_most_similar(x['adj'], tipo='ALL', n=20)), axis=1)

*4 rankings segun 4 denominadores --
pensar como combinar los denominadores (media, max?)*

In [0]:
deps_df['ranking_noun_adjs'] = deps_df['dist'] / deps_df['noun_avg_dist_adjs']
deps_df['ranking_adj_nouns'] = deps_df['dist'] / deps_df['adj_avg_dist_nouns']
deps_df['ranking_noun_words'] = deps_df['dist'] / deps_df['noun_avg_dist_words']
deps_df['ranking_adj_words'] = deps_df['dist'] / deps_df['adj_avg_dist_words']

*al final de todo: recuperar las oraciones de los n mas altos*

In [0]:
deps_df.sort_values(by=['ratio_head'], inplace=True, ascending=False)
df_top = deps_df.head(20).copy()
df_top['sentence'] = df_top['sent_idx'].apply(lambda x: docs.sentences[x].text)
df_bottom = deps_df.tail(20).copy()
df_bottom['sentence'] = df_bottom['sent_idx'].apply(lambda x: docs.sentences[x].text)

In [0]:
for i,j,k in zip(df_top['dep'], df_top['head'], df_top['sentence']):
    print(i,j,' -- ',k)

In [0]:
print(df_bottom[['dep','head','dist']])

<!-- *Cosas viejas*



lengths = [len(doc.sentences) for doc in docs]
for i in lengths:
    print(i)
# No siempre da 1!!! (pero se puede corregir el idx de abajo creo)
print(deps[len(deps)-1])
sentences[129]
print(deps[:5])
# funciona el rastreo de oraciones!! :)

# def words_distance(vocab, *words):
#     """ Get (cosine?) distance between 2 Glove word vectors if in vocabulary \ 
#         (dict_keys) 
#     Returns: distance (float)
#     """
#     if set(words).issubset(vocab):
#         d = word_vectors.distance(*words)
#     else:
#         d = np.nan
#     return d
# vocab = word_vectors.vocab.keys()
# data_dep = pd.DataFrame(deps, columns =['sent_idx', 'head', 'dep']) 
# data_dep['dist'] = data_dep.apply(lambda x: \
#                                   words_distance(vocab, x['head'], x['dep']), 
#                                   axis=1)

### OLD CODE

# def find_dependencies_v2(docs):
#     """Find amod dependencies in many parsed docs (idx of sentence with \
#     respect to all docs)
#     Returns: list of dependencies with (sentence_idx, head, dependent)
#     """
#     deps = list()
#     sent_idx = 0
#     for doc in docs:
#         for sent in doc.sentences:
#             id2word = {word.id: word.text for word in sent.words}
#             deps += [(sent_idx, id2word[str(word.head)], word.text) \
#                     for word in sent.words if word.deprel=='amod']
#         sent_idx += 1
#     return deps
# deps = find_dependencies(docs)

# def create_batches(sentences, sentences_per_doc=16):
#     """Creates batches of sentences
#        Returns: list of strings, each one containg multiple sentences separated
#        by \n\n 
#     """
#     ranges = [(max(0,i), min(i+sentences_per_doc,len(sentences))) \
#                  for i, x in enumerate(sentences) if \
#                  i % sentences_per_doc == 0]
#     batches = ['\n\n'.join(sentences[i:s]) for i, s in ranges]
#     return batches

# sentences = clean_texto(texto)
# batches = create_batches(sentences, sentences_per_doc=16)
# print('{} sentences found'.format(len(sentences)))
# print('{} batches created'.format(len(batches)))

# def parse_sentences(batches):
#     """Parse batched sentences 
#     Returns: a list of docs, each one cointaining multiple parsed sentences
#     """
#     docs = list()
#     for doc in batches:
#         docs.append(nlp(doc))
#     # ponemos los docs en una sola lista
#     docs = list(docs)
#     return docs
# # docs = parse_sentences(batches)
# docs = parse_sentences(sentences) -->